pip install prophet --quiet - встановіть через термінал.
pip install pandas
pip install matplotlib

Завантаження, та підготовка даних.

In [1]:
import pandas as pd
from prophet import Prophet

/Users/karina-zaika/Documents/business_forecasting_zaika/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
df = pd.read_csv('forecast_of_market_dataset.csv')

In [3]:
df['month'] = pd.to_datetime(df['month'], dayfirst=True)

Запуск моделі

In [4]:
def forecast_for_target(df, target_col):
    updated_rows = []

    for group in df['product_group_id'].unique():
        group_df = df[df['product_group_id'] == group].copy()

        # Separate historical and future
        hist = group_df[group_df[target_col].notna()][['month', target_col]].copy()
        future = group_df[group_df[target_col].isna()][['month']].copy()

        # Skip if not enough history
        if len(hist) < 6:
            continue

        # Rename for Prophet
        hist.rename(columns={'month': 'ds', target_col: 'y'}, inplace=True)

        # Train model
        model = Prophet()
        model.fit(hist)

        # Make future df
        future.rename(columns={'month': 'ds'}, inplace=True)
        forecast = model.predict(future)

        # Get only predicted values
        predicted = forecast[['ds', 'yhat']].copy()
        predicted.rename(columns={'ds': 'month', 'yhat': target_col}, inplace=True)
        predicted['product_group_id'] = group

        updated_rows.append(predicted)

    return pd.concat(updated_rows, ignore_index=True)

In [5]:
market_forecast = forecast_for_target(df, 'market_revenue')
amazon_forecast = forecast_for_target(df, 'revenue_amazon')

21:32:08 - cmdstanpy - INFO - Chain [1] start processing
21:32:09 - cmdstanpy - INFO - Chain [1] done processing
21:32:09 - cmdstanpy - INFO - Chain [1] start processing
21:32:09 - cmdstanpy - INFO - Chain [1] done processing
21:32:09 - cmdstanpy - INFO - Chain [1] start processing
21:32:10 - cmdstanpy - INFO - Chain [1] done processing
21:32:10 - cmdstanpy - INFO - Chain [1] start processing
21:32:10 - cmdstanpy - INFO - Chain [1] done processing
21:32:10 - cmdstanpy - INFO - Chain [1] start processing
21:32:10 - cmdstanpy - INFO - Chain [1] done processing
21:32:10 - cmdstanpy - INFO - Chain [1] start processing
21:32:10 - cmdstanpy - INFO - Chain [1] done processing
21:32:10 - cmdstanpy - INFO - Chain [1] start processing
21:32:10 - cmdstanpy - INFO - Chain [1] done processing
21:32:10 - cmdstanpy - INFO - Chain [1] start processing
21:32:10 - cmdstanpy - INFO - Chain [1] done processing
21:32:10 - cmdstanpy - INFO - Chain [1] start processing
21:32:10 - cmdstanpy - INFO - Chain [1]

Збереження результатів в Датасет.

In [6]:
df = df.merge(market_forecast, on=['month', 'product_group_id'], how='left', suffixes=('', '_forecasted'))
df = df.merge(amazon_forecast, on=['month', 'product_group_id'], how='left', suffixes=('', '_forecasted'))

In [7]:
df['market_revenue'] = df['market_revenue'].fillna(df['market_revenue_forecasted'])
df['revenue_amazon'] = df['revenue_amazon'].fillna(df['revenue_amazon_forecasted'])


In [8]:
df.drop(columns=['market_revenue_forecasted', 'revenue_amazon_forecasted'], inplace=True)

In [9]:
df.to_csv('forecasted_market_amazon_revenue.csv', index=False)

In [10]:
df[df['month'].dt.year == 2025].head()

,month,product_group_name,product_group_id,market_revenue,revenue_amazon
72,2025-01-01,Portable Computers,1,1.326556e+10,3.154167e+09
73,2025-01-02,Portable Computers,1,1.466801e+10,3.805620e+09
74,2025-01-03,Portable Computers,1,1.235090e+10,3.316072e+09
75,2025-01-04,Portable Computers,1,1.347644e+10,3.599457e+09
76,2025-01-05,Portable Computers,1,1.212867e+10,3.212528e+09
